In [5]:
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
from operator import itemgetter

import matplotlib.pyplot as plt
from sklearn import neighbors, datasets, metrics
from sklearn.model_selection import StratifiedKFold

from scipy.stats import zscore

from scipy import ndimage as ndi
import misc
import data_analysis_func as func


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [11]:
radius = 350
number_of_samples = 320                    # Number of samples to be validated in folds

#path_img = '3'
path_img = 'S07_46186 10X_20151028_59'
path = 'prostate_marked/' + path_img + '/results_radius_' + str(radius) + '/'

G_read = nx.read_gml(path + 'grafo_glands_properties_' + str(radius) + 'r.gml')
nodes, nodes_demarcated = func.display_gland_numbers(G_read)

Number of NORMAL glands:  2240
Number of DEMARCATED glands:  1124


# Cross Validation

In [12]:
k = 3                                       # Number of neighbors for the k-nearest neighbors classifier
n_classes = 2                               # Number of classes in the dataset
n_splits = 5                                # Number of Folds

print_classes = False                       # Option to visualize fold data class
print_all_accuracy = True                   # Option to visualize fold by fold accuracy


#Properties to be validated
# Shapes  - 1
# Network - 2
# All     - 3
# All idx - 4

props_type = 4
properties_names = None



# Shapes props
if props_type == 1:
    properties_names = ['area', 'diameter', 'perimeter', 'eccentricity', 'solidity']

# Network props
elif props_type == 2:
    properties_names = ['degree', 'betweenness', 'strength']

# Shapes and Network props
elif props_type == 3:
    properties_names = ['area', 'diameter', 'perimeter', 'eccentricity', 'solidity', 'degree', 'betweenness', 'strength']

# All Props with IDX
else:
    properties_names = ['idx', 'area', 'diameter', 'perimeter', 'eccentricity', 'solidity', 'degree', 'betweenness', 'strength']    

table_properties_norm = func.get_table_properties(nodes, nodes_demarcated, properties_names, number_of_samples, True, False)

# Class target to Kfold
n_target = np.array([0]*number_of_samples + [1]*number_of_samples)

TypeError: get_table_properties() takes from 2 to 3 positional arguments but 6 were given

In [ ]:
table_properties_norm = func.remove_idx_from_table_properties(properties_names, table_properties_norm)

In [ ]:
X = table_properties_norm
y = n_target
acc_sum = 0

skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)


list_test_index = []
list_classes_pred = []

for train_index, test_index in skf.split(X, y):
   
    list_test_index.append(test_index)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    properties_train = X_train
    properties_validate = X_test
    classes_train = y_train
    classes_validate = y_test
    
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(properties_train, classes_train)

    # Predict the class using the validation set
    classes_pred = clf.predict(properties_validate)

    list_classes_pred.append(classes_pred)
    
    # Get accuracy
    acc = metrics.accuracy_score(classes_validate, classes_pred)
    acc_sum = acc_sum + acc
    
    if(print_classes):
        print("TRAIN:", train_index, "TEST:", test_index)
        print("X_train:",X_train)
        print("X_test:", X_test)
        print("y_train:", y_train)
        print("y_test", y_test)
    
    if(print_all_accuracy):
        print("Accuracy: ", acc)
        
print("")
print(properties_names)
print("\nMean: " + str(acc_sum/n_splits))

# Colorize Glands

In [ ]:
img_to_colorize = 'prostate_marked/' + path_img + '/MASK - RGB.png'
img = plt.imread(img_to_colorize)
misc.show_img(img)

img = 255-(255*plt.imread(img_to_colorize)[:,:,0]).astype(np.uint8)
misc.show_img(img)

In [ ]:
# Get Color objects color GREEN (not demarcated) and color RED (demarcated) otherwise ORANGE(undefined)
position, class_colors, measurements = func.get_color_inputs(G_read)

In [ ]:
# Get Color objects according to the colors defined in `class_colors`
img_colored1 = func.color_objects(img, position, class_colors, print_progress=True)

In [ ]:
plt.figure(figsize=(30,15))
plt.imshow(img_colored1)
plt.savefig(path + 'results_colorize/img_colored_two_classes.png')

In [ ]:
list_position = []
list_class_colors = []
list_measurements = []
list_confusion_matrix = []

for idx_list in range(len(list_classes_pred)):
    position, class_colors, measurements, confusion_matrix = func.get_color_inputs_by_cross_results(G_read, list_classes_pred[idx_list], list_test_index[idx_list], idx_selected)
    list_position.append(position)
    list_class_colors.append(class_colors)
    list_measurements.append(measurements)
    list_confusion_matrix.append(confusion_matrix)

In [ ]:
true_negative = 0
false_negative = 0
false_positive = 0
true_positive = 0
for i in range(len(list_confusion_matrix)):
    true_negative += list_confusion_matrix[i][0][0]
    false_negative += list_confusion_matrix[i][0][1]
    false_positive += list_confusion_matrix[i][1][0]
    true_positive += list_confusion_matrix[i][1][1]

mean_confusion_matrix = [ [true_negative, false_negative],
                          [false_positive, true_positive] ]
print (mean_confusion_matrix)
print(np.matrix(mean_confusion_matrix).sum())

In [ ]:
imgs_colored_cross = []
for idx_list in range(len(list_classes_pred)):
    colored_cross = func.color_objects(img, list_position[idx_list], list_class_colors[idx_list], print_progress=False)
    imgs_colored_cross.append(colored_cross)
    print(idx_list)    

In [ ]:
for idx, img_cross in enumerate(imgs_colored_cross):
    plt.figure(figsize=(30,15))
    plt.imshow(img_cross)
    plt.savefig(path + 'results_colorize_' + str(props_type) + '/img_colored_cross_idx_' + str(idx) + '.png')

In [ ]:
# Color objects according to the values contained in `measurements`
imgs_colored_cross = []
for idx_list in range(len(list_classes_pred)):
    colored_cross = func.color_objects(img, list_position[idx_list], values=list_measurements[idx_list], colormap='viridis', print_progress=False)
    imgs_colored_cross.append(colored_cross)
    print(idx_list)    

In [ ]:
for idx, img_cross in enumerate(imgs_colored_cross):
    plt.figure(figsize=(30,15))
    plt.imshow(img_cross)
    plt.savefig(path + 'results_colorize_' + str(props_type) + '/img_colored_cross_measurements_idx_' + str(idx) + '.png')